In [0]:
from pyspark.sql import SparkSession
import boto3
import os

# Iniciando uma SparkSession com Delta Lake
spark = SparkSession.builder \
    .appName("DeltaLakeSparkS3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Adicionando configurações S3 diretamente na sessão Spark
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", os.getenv("AWS_SECRET_KEY"))
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configurações dos buckets S3
landing_bucket = "engenharia-dados-satc-landing-zone-bucket"
bronze_bucket = "engenharia-dados-satc-bronze-bucket"

# Verificando os arquivos no bucket S3
s3 = boto3.client('s3', aws_access_key_id=os.getenv("AWS_ACCESS_KEY"), aws_secret_access_key=os.getenv("AWS_SECRET_KEY"))
response = s3.list_objects_v2(Bucket=landing_bucket)

print("Arquivos no bucket S3:")
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("Nenhum arquivo encontrado no bucket especificado.")

# Função para ler dados do S3
def read_from_s3(bucket, path):
    try:
        df = spark.read.format("csv").option("header", "true").load(f"s3a://{bucket}/{path}")
        return df
    except Exception as e:
        print(f"Erro ao ler dados do S3: {e}")

# Função principal
def main():
    try:
        # Definindo os caminhos dos arquivos CSV na landing
        paths = ['dadosprodutos.csv', 'pedidos.csv', 'clientes.csv', 'vendedores.csv', 'pagamentos.csv', 'dadospedidos.csv']
        dfs = {}
        for path in paths:
            df_name = path.split('.')[0]
            dfs[df_name] = read_from_s3(landing_bucket, f"/{path}")
        
        dfs["localizacoes"] = read_from_s3(landing_bucket, f"/geo.csv")

        # Salvando os DataFrames no formato Parquet usando Delta Lake no bucket da camada bronze
        for df_name, df in dfs.items():
            df.write.format("delta").mode('overwrite').save(f"s3a://{bronze_bucket}/{df_name}")
        
        print("Dados salvos na camada bronze com sucesso.")
        
    except Exception as e:
        print(f"Erro no processo principal: {e}")

if __name__ == "__main__":
    main()


Arquivos no bucket S3:
clientes.csv/_SUCCESS
clientes.csv/_committed_9112049085076660698
clientes.csv/_started_9112049085076660698
clientes.csv/part-00000-tid-9112049085076660698-6d781d67-96b0-4965-93d1-6404a6992252-1-1-c000.csv
dadospedidos.csv/_SUCCESS
dadospedidos.csv/_committed_2411623796930298435
dadospedidos.csv/_started_2411623796930298435
dadospedidos.csv/part-00000-tid-2411623796930298435-5e58a814-021a-4f6a-988d-8f0047b6d786-4-1-c000.csv
dadosprodutos.csv/_SUCCESS
dadosprodutos.csv/_committed_5449724296839749973
dadosprodutos.csv/_started_5449724296839749973
dadosprodutos.csv/part-00000-tid-5449724296839749973-cb4154ad-ea1b-4619-a844-031207ee518c-6-1-c000.csv
geo.csv/_SUCCESS
geo.csv/_committed_8630313209848014780
geo.csv/_started_8630313209848014780
geo.csv/part-00000-tid-8630313209848014780-0f3f4cfd-9f37-47a0-8321-f8c8c2209ff2-0-1-c000.csv
pagamentos.csv/_SUCCESS
pagamentos.csv/_committed_5853869466763043177
pagamentos.csv/_started_5853869466763043177
pagamentos.csv/part-000